This notebook explains the bais in the data for RACE with respoect to Frisk. Below are the steps to check the bais in the data or not:
1. Group races into privileged & unprivileged categorie
2. Compute fairness metrics for each racial group
    * Chi Square test
    * Disparate impact
    * Statistical parity
    * Equal opportunity difference


In [2]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [3]:
years = [2017,2018,2019,2020,2021,2022,2023]

files=['sqf-2017.xlsx',
 'sqf-2018.xlsx',
 'sqf-2019.xlsx',
 'sqf-2020.xlsx',
 'sqf-2021.xlsx',
 'sqf-2022.xlsx',
 'sqf-2023.xlsx',
 ]

df = pd.read_excel('./sqf-2024.xlsx')
for file,year in zip(files,years):
    df_year = pd.read_excel(file)
    df_year['year'] = year
    cols = [col for col in df.columns if col in df_year.columns]
    print(f"{year} : {df_year.shape}")
    df = pd.concat([df,df_year[cols]])

2017 : (11629, 84)
2018 : (11008, 84)
2019 : (13459, 84)
2020 : (9544, 84)
2021 : (8947, 84)
2022 : (15102, 83)
2023 : (16971, 83)


In [4]:
RACE = 'SUSPECT_RACE_DESCRIPTION'
target = 'FRISKED_FLAG'

In [5]:
df[target].value_counts()

,count
FRISKED_FLAG,
Y,67213
N,44832
V,1


In [66]:
df_filter = df[~df.SUSPECT_RACE_DESCRIPTION.isin(['(null)','MALE'])]
df_filter.shape,df.shape

((109807, 81), (112046, 81))

In [67]:
df_filter['SUSPECT_RACE_DESCRIPTION'] = np.where(df_filter['SUSPECT_RACE_DESCRIPTION']=='ASIAN/PAC.ISL','ASIAN / PACIFIC ISLANDER',df_filter['SUSPECT_RACE_DESCRIPTION'])
df_filter['SUSPECT_RACE_DESCRIPTION'] = np.where(df_filter['SUSPECT_RACE_DESCRIPTION']=='MIDDLE EASTERN/SOUTHWEST','MIDDLE EASTERN/SOUTHWEST ASIAN',df_filter['SUSPECT_RACE_DESCRIPTION'])
df_filter['SUSPECT_RACE_DESCRIPTION'] = np.where(df_filter['SUSPECT_RACE_DESCRIPTION']=='AMERICAN INDIAN/ALASKAN N','AMERICAN INDIAN/ALASKAN NATIVE',df_filter['SUSPECT_RACE_DESCRIPTION'])
df_filter['SUSPECT_RACE_DESCRIPTION'] = np.where(df_filter['SUSPECT_RACE_DESCRIPTION']=='AMER IND','AMERICAN INDIAN/ALASKAN NATIVE',df_filter['SUSPECT_RACE_DESCRIPTION'])


In [68]:
df_filter['SUSPECT_RACE_DESCRIPTION'].value_counts(normalize=True)

,proportion
SUSPECT_RACE_DESCRIPTION,
BLACK,0.592421
WHITE HISPANIC,0.210788
BLACK HISPANIC,0.093983
WHITE,0.075041
ASIAN / PACIFIC ISLANDER,0.019871
MIDDLE EASTERN/SOUTHWEST ASIAN,0.006566
AMERICAN INDIAN/ALASKAN NATIVE,0.001330


In [69]:
feats=['BACKROUND_CIRCUMSTANCES_VIOLENT_CRIME_FLAG',
       'SUSPECTED_CRIME_DESCRIPTION',
       'SUSPECTS_ACTIONS_CONCEALED_POSSESSION_WEAPON_FLAG','STOP_LOCATION_X','STOP_LOCATION_Y',
       'STOP_WAS_INITIATED','STOP_DURATION_MINUTES','STOP_FRISK_TIME','SUSPECT_REPORTED_AGE',
       'SUSPECT_HEIGHT','SUSPECT_WEIGHT','SUSPECT_SEX',
       'SUSPECT_RACE_DESCRIPTION','SEARCHED_FLAG','SEARCH_BASIS_HARD_OBJECT_FLAG',
       'CONSENT_GIVEN_FLG','FRISKED_FLAG']

In [70]:
df_filter = df_filter[df_filter.STOP_WAS_INITIATED!='(null)']
df_filter['BACKROUND_CIRCUMSTANCES_VIOLENT_CRIME_FLAG'] = np.where(df_filter.BACKROUND_CIRCUMSTANCES_VIOLENT_CRIME_FLAG=='(null)',0,1)
df_filter['SUSPECTS_ACTIONS_CONCEALED_POSSESSION_WEAPON_FLAG'] = np.where(df_filter.SUSPECTS_ACTIONS_CONCEALED_POSSESSION_WEAPON_FLAG=='(null)',0,1)
df_filter['SUSPECT_SEX'] = np.where(df_filter.SUSPECT_SEX=='(null)','UNKNOWN',df_filter.SUSPECT_SEX)
df_filter['SEARCHED_FLAG'] = np.where(df_filter.SEARCHED_FLAG=='Y',1,0)
df_filter['SEARCH_BASIS_HARD_OBJECT_FLAG'] = np.where(df_filter.SEARCH_BASIS_HARD_OBJECT_FLAG=='(null)',0,1)

df_filter['CONSENT_GIVEN_FLG'] = np.where(df_filter.CONSENT_GIVEN_FLG.isin(['(null)','(']),'U',df_filter.CONSENT_GIVEN_FLG)
df_filter['FRISKED_FLAG'] = np.where(df_filter.FRISKED_FLAG=='Y',1,0)

df_filter['SUSPECT_HEIGHT'] = np.where(df_filter.SUSPECT_HEIGHT.isin(['(null)',' ']),None,df_filter.SUSPECT_HEIGHT)
df_filter['SUSPECT_HEIGHT'] = df_filter.SUSPECT_HEIGHT.astype(float)
df_filter['SUSPECT_HEIGHT'] = df_filter.SUSPECT_HEIGHT.fillna(df_filter.SUSPECT_HEIGHT.mean())

df_filter['SUSPECT_WEIGHT'] = np.where(df_filter.SUSPECT_WEIGHT.isin(['(null)',' ']),None,df_filter.SUSPECT_WEIGHT)
df_filter['SUSPECT_WEIGHT'] = df_filter.SUSPECT_WEIGHT.astype(float)
df_filter['SUSPECT_WEIGHT'] = df_filter.SUSPECT_WEIGHT.fillna(df_filter.SUSPECT_WEIGHT.mean())

df_filter['SUSPECT_REPORTED_AGE'] = np.where(df_filter.SUSPECT_REPORTED_AGE.isin(['(null)',' ']),None,df_filter.SUSPECT_REPORTED_AGE)
df_filter['SUSPECT_REPORTED_AGE'] = df_filter.SUSPECT_REPORTED_AGE.astype(float)
df_filter['SUSPECT_REPORTED_AGE'] = df_filter.SUSPECT_REPORTED_AGE.fillna(df_filter.SUSPECT_REPORTED_AGE.mean())



#### Check Bias across multiple races

The **Chi-Square Test of Independence** is used to determine whether there is a statistically significant relationship between two categorical variables—in this case, race and frisking outcomes. It helps identify potential bias in stop-and-frisk practices.

##### Methodology:
1. Construct a contingency table that records the number of frisked and non-frisked individuals across different racial groups.
2. Compute the Chi-Square statistic using the formula:<br>
$$χ^{2} = \sum \frac{(O - E)^{2}}{E} $$

 where:

  * O = observed frequency in each category

  * E = expected frequency if there were no racial bias

3. Obtain the **p-value** to test the null hypothesis (H₀: Frisking is independent of race).
4. Compare the p-value with a significance level (𝛼=0.05):
  * p < 0.05 → Reject H₀ → Indicates a significant racial disparity (potential bias).
  * p ≥ 0.05 → Fail to reject H₀ → No strong evidence of bias.

##### Results Interpretation:
* A **significant Chi-Square** result suggests racial bias in frisking, indicating that race influences frisking outcomes.
* If the test does not find a significant difference, it means frisking is distributed proportionally across racial groups.

This test provides an initial statistical check for bias, which is further validated using fairness metrics like Equal Opportunity Difference, Disparate Impact, and Statistical Parity Difference. If bias is detected, bias mitigation techniques (such as reweighing or adversarial debiasing) are applied to ensure fairness in predictive models.



In [72]:
import scipy.stats as stats
# Creating a contingency table
contingency_table = pd.crosstab(df_filter['SUSPECT_RACE_DESCRIPTION'], df_filter['FRISKED_FLAG'])

# Perform the chi-square test
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

# Display results
print("Chi-square Statistic:", chi2)
print("P-value:", p)
print("Degrees of Freedom:", dof)
print("Expected Frequencies:\n", expected)

# Interpretation
alpha = 0.05  # Significance level
if p < alpha:
    print("There is a statistically significant relationship between RACE and FRISKED. Potential bias detected.")
else:
    print("No statistically significant relationship found. No strong evidence of bias in frisking.")


Chi-square Statistic: 2115.8903256843155
P-value: 0.0
Degrees of Freedom: 6
Expected Frequencies:
 [[   58.94174545    87.05825455]
 [  880.89649708  1301.10350292]
 [26260.56615116 38787.43384884]
 [ 4164.67839773  6151.32160227]
 [  291.07533199   429.92466801]
 [ 3326.1715121   4912.8284879 ]
 [ 9342.6703645  13799.3296355 ]]
There is a statistically significant relationship between RACE and FRISKED. Potential bias detected.


To evaluate potential racial bias in frisking outcomes, we use three key fairness metrics: Equal Opportunity Difference (EOD), Disparate Impact (DI), and Statistical Parity Difference (SPD). These metrics help quantify disparities between privileged and unprivileged racial groups.

##### 1. Equal Opportunity Difference (EOD)
**Definition:** Measures the difference in frisking rates between unprivileged and privileged groups, conditioned on positive outcomes (i.e., individuals who should have been frisked).

$EOD=P(Frisked∣Unprivileged Group, Should Be Frisked)−P(Frisked∣Privileged Group, Should Be Frisked)$

**Interpretation:**
* EOD = 0 → No disparity; both groups have equal frisking rates for relevant cases.
* EOD > 0 → Unprivileged group is frisked more than the privileged group.
* EOD < 0 → Unprivileged group is frisked less than the privileged group (potential under-policing).
* A large deviation from zero indicates possible discrimination.

##### 2. Disparate Impact (DI)
**Definition:** Measures the ratio of frisking rates between the unprivileged and privileged groups.

$$DI= \frac{P(Frisked∣Unprivileged Group)}{P(Frisked∣Privileged Group)}$$

**Interpretation:**
* DI = 1 → No bias; both groups have equal frisking rates.
* DI < 0.8 → Strong evidence of bias against the unprivileged group (per U.S. Equal Employment Opportunity Commission guidelines).
* DI > 1.25 → Potential bias in favor of the unprivileged group.

##### 3. Statistical Parity Difference (SPD)
**Definition:** Measures the difference in overall frisking rates between unprivileged and privileged groups.

$$SPD=P(Frisked∣Unprivileged Group)−P(Frisked∣Privileged Group)$$

**Interpretation:**
* SPD = 0 → No disparity.
* SPD > 0 → Unprivileged group is frisked more than the privileged group.
* SPD < 0 → Unprivileged group is frisked less than the privileged group.
* A larger absolute value suggests potential bias.

These metrics provide a deeper understanding of how frisking outcomes differ across racial groups. If significant bias is detected, mitigation strategies such as Reweighing (Pre-processing) and Adversarial Debiasing (In-processing) are applied to improve fairness while maintaining model performance.



In [73]:
# Bias Metrics Calculation
def calculate_bias_metrics(df, privileged_race):
    race_groups = df['SUSPECT_RACE_DESCRIPTION'].unique()

    rates = {}
    for race in race_groups:
        frisked_rate = df[df['SUSPECT_RACE_DESCRIPTION'] == race]['FRISKED_FLAG'].mean()
        rates[race] = frisked_rate

    privileged_rate = rates[privileged_race]
    print(f"privileged_rate : {privileged_rate}")
    print(f"rates : {rates}")

    # Equal Opportunity Difference (EOD)
    eod = {race: rates[race] - privileged_rate for race in rates}

    # Disparate Impact (DI)
    di = {race: rates[race] / privileged_rate if privileged_rate > 0 else None for race in rates}

    # Statistical Parity Difference (SPD)
    spd = eod.copy()

    return eod, di, spd

# Choose a privileged race (modify as needed)
privileged_race = 'WHITE HISPANIC'

eod, di, spd = calculate_bias_metrics(df_filter, privileged_race)

print("\nEqual Opportunity Difference:", eod)
print("Disparate Impact:", di)
print("Statistical Parity Difference:", spd)

# Bias Interpretation
bias_threshold = 0.8  # Disparate impact threshold
for race in di:
    if di[race] is not None and di[race] < bias_threshold:
        print(f"Potential bias detected against {race} with Disparate Impact below threshold.")


privileged_rate : 0.5790337913749892
rates : {'WHITE HISPANIC': np.float64(0.5790337913749892), 'BLACK': np.float64(0.6296427253720329), 'BLACK HISPANIC': np.float64(0.6308646762310973), 'WHITE': np.float64(0.3846340575312538), 'MIDDLE EASTERN/SOUTHWEST ASIAN': np.float64(0.47295423023578365), 'ASIAN / PACIFIC ISLANDER': np.float64(0.4738771769019248), 'AMERICAN INDIAN/ALASKAN NATIVE': np.float64(0.410958904109589)}

Equal Opportunity Difference: {'WHITE HISPANIC': np.float64(0.0), 'BLACK': np.float64(0.05060893399704369), 'BLACK HISPANIC': np.float64(0.051830884856108095), 'WHITE': np.float64(-0.19439973384373543), 'MIDDLE EASTERN/SOUTHWEST ASIAN': np.float64(-0.10607956113920558), 'ASIAN / PACIFIC ISLANDER': np.float64(-0.1051566144730644), 'AMERICAN INDIAN/ALASKAN NATIVE': np.float64(-0.1680748872654002)}
Disparate Impact: {'WHITE HISPANIC': np.float64(1.0), 'BLACK': np.float64(1.0874023843701184), 'BLACK HISPANIC': np.float64(1.089512711741795), 'WHITE': np.float64(0.66426875816330

In [74]:
# !pip install aif360
# !pip install aif360[Reductions]
# !pip install aif360[AdversarialDebiasing]
# !pip install aif360[inFairness]
# !pip install tensorflow

In [15]:
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.datasets import StandardDataset
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf
#from sklearn.preprocessing import OneHotEncoder, StandardScaler
#from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression

In [75]:
req_feats=['BACKROUND_CIRCUMSTANCES_VIOLENT_CRIME_FLAG',
       'SUSPECTED_CRIME_DESCRIPTION',
       'SUSPECTS_ACTIONS_CONCEALED_POSSESSION_WEAPON_FLAG',
       'STOP_WAS_INITIATED','STOP_DURATION_MINUTES','STOP_FRISK_TIME','SUSPECT_REPORTED_AGE',
       'SUSPECT_HEIGHT','SUSPECT_WEIGHT','SUSPECT_SEX',
       'SUSPECT_RACE_DESCRIPTION','SEARCHED_FLAG','SEARCH_BASIS_HARD_OBJECT_FLAG',
       'CONSENT_GIVEN_FLG','FRISKED_FLAG']
cat_feats = ['STOP_WAS_INITIATED','SUSPECT_SEX',
             'CONSENT_GIVEN_FLG','SUSPECTED_CRIME_DESCRIPTION']

df_filter = df_filter[req_feats]

df_filter = pd.get_dummies(df_filter,columns=cat_feats)
df_filter.head()

,BACKROUND_CIRCUMSTANCES_VIOLENT_CRIME_FLAG,SUSPECTS_ACTIONS_CONCEALED_POSSESSION_WEAPON_FLAG,STOP_DURATION_MINUTES,STOP_FRISK_TIME,SUSPECT_REPORTED_AGE,SUSPECT_HEIGHT,SUSPECT_WEIGHT,SUSPECT_RACE_DESCRIPTION,SEARCHED_FLAG,SEARCH_BASIS_HARD_OBJECT_FLAG,...,SUSPECTED_CRIME_DESCRIPTION_MURDER,SUSPECTED_CRIME_DESCRIPTION_OTHER,SUSPECTED_CRIME_DESCRIPTION_PETIT LARCENY,SUSPECTED_CRIME_DESCRIPTION_PROSTITUTION,SUSPECTED_CRIME_DESCRIPTION_RAPE,SUSPECTED_CRIME_DESCRIPTION_RECKLESS ENDANGERMENT,SUSPECTED_CRIME_DESCRIPTION_ROBBERY,SUSPECTED_CRIME_DESCRIPTION_TERRORISM,SUSPECTED_CRIME_DESCRIPTION_THEFT OF SERVICES,SUSPECTED_CRIME_DESCRIPTION_UNAUTHORIZED USE OF A VEHICLE
0,0,1,1,01:58:00,27.0,5.11,160.0,WHITE HISPANIC,1,1,...,False,False,False,False,False,False,False,False,False,False
1,0,1,3,00:48:00,22.0,6.10,200.0,BLACK,1,1,...,False,False,False,False,False,False,False,False,False,False
2,0,0,20,01:10:00,17.0,6.00,140.0,WHITE HISPANIC,0,0,...,False,False,False,False,False,False,False,False,False,False
3,0,0,20,01:10:00,13.0,5.00,120.0,WHITE HISPANIC,0,0,...,False,False,False,False,False,False,False,False,False,False
4,0,0,20,01:10:00,14.0,5.00,120.0,WHITE HISPANIC,0,0,...,False,False,False,False,False,False,False,False,False,False


In [76]:
#df_filter.STOP_FRISK_TIME.value_counts()

time_cat = pd.to_datetime(df_filter['STOP_FRISK_TIME'], errors='coerce').dt.time
# Function to categorize time
def categorize_time(t):
    if pd.isna(t):
        return 'Unknown'  # Or use 'NaN' / 'Missing'
    elif t >= pd.to_datetime('05:00', format='%H:%M').time() and t < pd.to_datetime('12:00', format='%H:%M').time():
        return 'Morning'
    elif t >= pd.to_datetime('12:00', format='%H:%M').time() and t < pd.to_datetime('17:00', format='%H:%M').time():
        return 'Afternoon'
    elif t >= pd.to_datetime('17:00', format='%H:%M').time() and t < pd.to_datetime('21:00', format='%H:%M').time():
        return 'Evening'
    else:
        return 'Night'

#df['time_of_day'] =
df_filter['time_of_day'] = time_cat.apply(categorize_time)
df_filter = pd.get_dummies(df_filter,columns=['time_of_day'])
df_filter = df_filter.drop(columns=['STOP_FRISK_TIME'])

In [78]:
# Bias Mitigation using Reweighing from AIF360
sensitive_attribute = 'SUSPECT_RACE_DESCRIPTION'
target_attribute = 'FRISKED_FLAG'

dataset = StandardDataset(df_filter,
                          label_name=target_attribute,
                          favorable_classes=[1],
                          protected_attribute_names=[sensitive_attribute],
                          privileged_classes=[['WHITE']])

In [80]:
# Apply Reweighing
reweighing = Reweighing(unprivileged_groups=[{sensitive_attribute: race} for race in df['SUSPECT_RACE_DESCRIPTION'].unique() if race != 'WHITE'],
                        privileged_groups=[{sensitive_attribute: 'WHITE'}])
reweighed_dataset = reweighing.fit_transform(dataset)

In [82]:
# Assign new weights to the dataset
df_filter['sample_weight'] = reweighed_dataset.instance_weights

print("Bias mitigation applied using Reweighing method. The dataset now includes reweighted sample weights.")


Bias mitigation applied using Reweighing method. The dataset now includes reweighted sample weights.


In [86]:
# In-Processing Bias Mitigation using Adversarial Debiasing
tf.compat.v1.disable_eager_execution()
unprivileged_race = 'BLACK'  # Modify based on analysis
adversarial_debiasing = AdversarialDebiasing(
            privileged_groups=[{sensitive_attribute: 'WHITE'}],
            unprivileged_groups=[{sensitive_attribute: unprivileged_race}],
            scope_name='adv_debiasing',
            sess=tf.compat.v1.Session(),
            debias=True
        )
adversarial_debiasing.fit(dataset)
mitigated_predictions = adversarial_debiasing.predict(dataset)

print("Bias mitigation applied using Adversarial Debiasing (In-Processing). The model is trained to reduce bias.")


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 3.311165; batch adversarial loss: 1.030446
epoch 0; iter: 200; batch classifier loss: 0.683279; batch adversarial loss: 0.766853
epoch 0; iter: 400; batch classifier loss: 0.521977; batch adversarial loss: 0.538314
epoch 0; iter: 600; batch classifier loss: 0.507458; batch adversarial loss: 0.476089
epoch 0; iter: 800; batch classifier loss: 0.636375; batch adversarial loss: 0.433462
epoch 1; iter: 0; batch classifier loss: 0.600192; batch adversarial loss: 0.364091
epoch 1; iter: 200; batch classifier loss: 0.802412; batch adversarial loss: 0.408633
epoch 1; iter: 400; batch classifier loss: 0.724761; batch adversarial loss: 0.349326
epoch 1; iter: 600; batch classifier loss: 0.601935; batch adversarial loss: 0.326077
epoch 1; iter: 800; batch classifier loss: 0.508436; batch adversarial loss: 0.259955
epoch 2; iter: 0; batch classifier loss: 0.394206; batch adversarial loss: 0.225049
epoch 2; iter: 200; batch classifier loss: 0.464776; batch a

In [92]:
# Compare Models Before and After Debiasing
original_predictions = dataset.labels
#mitigated_predictions = mitigated_predictions.labels

# Performance Metrics
accuracy_before = accuracy_score(original_predictions, df_filter['FRISKED_FLAG'])
accuracy_after = accuracy_score(mitigated_predictions, df_filter['FRISKED_FLAG'])
f1_before = f1_score(original_predictions, df_filter['FRISKED_FLAG'])
f1_after = f1_score(mitigated_predictions, df_filter['FRISKED_FLAG'])

print("\nModel Performance Comparison:")
print(f"Accuracy Before Debiasing: {accuracy_before:.4f}")
print(f"Accuracy After Debiasing: {accuracy_after:.4f}")
print(f"F1 Score Before Debiasing: {f1_before:.4f}")
print(f"F1 Score After Debiasing: {f1_after:.4f}")

# Fairness Metrics Comparison
eod_after, di_after, spd_after = calculate_bias_metrics(df_filter, privileged_race)

print("\nFairness Metrics Comparison:")
print("Equal Opportunity Difference Before:", eod)
print("Equal Opportunity Difference After:", eod_after)
print("Disparate Impact Before:", di)
print("Disparate Impact After:", di_after)
print("Statistical Parity Difference Before:", spd)
print("Statistical Parity Difference After:", spd_after)


Model Performance Comparison:
Accuracy Before Debiasing: 1.0000
Accuracy After Debiasing: 0.8607
F1 Score Before Debiasing: 1.0000
F1 Score After Debiasing: 0.8825
privileged_rate : 0.5790337913749892
rates : {'WHITE HISPANIC': np.float64(0.5790337913749892), 'BLACK': np.float64(0.6296427253720329), 'BLACK HISPANIC': np.float64(0.6308646762310973), 'WHITE': np.float64(0.3846340575312538), 'MIDDLE EASTERN/SOUTHWEST ASIAN': np.float64(0.47295423023578365), 'ASIAN / PACIFIC ISLANDER': np.float64(0.4738771769019248), 'AMERICAN INDIAN/ALASKAN NATIVE': np.float64(0.410958904109589)}

Fairness Metrics Comparison:
Equal Opportunity Difference Before: {'WHITE HISPANIC': np.float64(0.0), 'BLACK': np.float64(0.05060893399704369), 'BLACK HISPANIC': np.float64(0.051830884856108095), 'WHITE': np.float64(-0.19439973384373543), 'MIDDLE EASTERN/SOUTHWEST ASIAN': np.float64(-0.10607956113920558), 'ASIAN / PACIFIC ISLANDER': np.float64(-0.1051566144730644), 'AMERICAN INDIAN/ALASKAN NATIVE': np.float64(-